# Problem Set 2 — Coding Part

**Lecture:** Data Compression With Deep probabilistic models (Prof. Bamler at University of Tuebingen)

- This notebook constitutes the coding part of Problem Set 2, published on 27 April 2021 and discussed on 3 May 2021.
- Download the full problem set (and solutions, once available) from the [course website](https://robamler.github.io/teaching/compress21/).

## Problem 2.4 (d): Huffman Coding

In this exercise, we'll implement the Huffman Coding algorithm (Algorithm 2 on the problem set).

We represent bit strings (code words) as lists of boolean values, where `True` represents a "one"-bit and `False` represents a "zero" bit.
Please be aware that this would be an extremely inefficient representation for a real application, but the simplicity of this representation will make it easier to understand the algorithm.

### Proposed Representation of the Huffman Tree

The following is a proposal for how you could represent the Huffman tree.
Feel free to use your own representation if you have a better idea.
This proposal is optimized for *encoding* because it stores only pointers from nodes to their parents.
If you want to use the Huffman tree for efficient *decoding* then you should also store the inverse pointers from nodes to their children.

A Huffman tree represents a code book as a binary tree with the symbols $x \in \mathfrak X = \{0,1,\ldots, |\mathfrak X|-1\}$ at its leaves.
Any binary tree with $|\mathfrak X|$ leaves has a total of $(2|\mathfrak X| - 1)$ nodes (including the leaves), regardless of the specific shape of the tree.
We represent the tree as a list (named `parent_nodes`) of length $(2|\mathfrak X| - 1)$ with the following properties:
- the first $|\mathfrak X|$ entries of the list `parent_nodes` correspond to the leaf nodes, i.e., the symbols $x\in\{0,1,\ldots,|\mathfrak X|-1\}$;
- the remaining $(|\mathfrak X|-1)$ entries of the list `parent_nodes` correspond to non-leaf nodes, with the root node at the very end;
- each entry of the list `parent_nodes` is a tuple `(index, label)` where `index` is an index into `parent_nodes` pointing at the parent of the current node, and `label` is the label of the edge between the node and its parent (which is either `True` or `False`);
- since the root node has no parent, `parent_nodes[len(parent_nodes) - 1]` will always be `None`.

### Your Tasks

- Read and understand the code in the method `encode_symbol`.
  Then complete the constructor (`__init__`) so that it creates the tree structure that `encode_symbol` expects.
- Run the minimal provided unit test to make sure you didn't do any obvious mistake.
- Implement and run additional unit tests. In particular, test edge case like ties or a degenerate probability distribution (that puts all mass on a single symbol.

In [1]:
import numpy as np
import heapq # See Problem 1.3 from last week's problem set and its code example.

In [2]:
class Huffman:
    """A code book for Huffman coding"""
    
    def __init__(self, probabilities):
        """Create an optimal prefix code using the Huffman algorithm.
        
        The alphabet is assumed to be of the form
        {0, 1, 2, ..., probabilities.len() - 1}.

        Args:
            probabilities (list or numpy array): The probabilities of each symbol
                in the alphabet. The first entry is the probability of symbol `0`,
                the second entry is the probability of symbol `1`, and so on.
                Must be nonnegative and sum to 1 (up to rounding errors) for the
                Huffman algorithm to be correct.
        """
        
        parent_nodes = [None]* ((2 * len(probabilities))-1)

        # first_nodes = [(value, probability) for value, probability in enumerate(probabilities)]
        # parent_nodes[0:len(probabilities)] = first_nodes
        # print(first_nodes)
        
        roots = []
        for probability in probabilities:
            heapq.heappush(roots, probability)
        # TODO: turn `roots` into a binary heap of pairs `(probability, symbol)`,
        # representing the set `R` from Algorithm 2 on the problem set.
        # (You may want to refer back to the code examples for Problem 1.3 from
        # last week's problem set.
        new_y = len(probabilities)-1
        while len(roots) > 1:
            w_1 = heapq.heappop(roots)
            y_1 = probabilities.index(w_1)
            w_2 = heapq.heappop(roots)
            y_2 = probabilities.index(w_2)
            new_y += 1
            new_w = w_1 + w_2

            parent_nodes[y_1] = (new_y, False)
            parent_nodes[y_2] = (new_y, True)
            heapq.heappush(roots, new_w)
            probabilities.append(new_w)



            # TODO: implement the rest of the algorithm, as described on the problem set:
            # - update the `roots` heap by popping of two items and pushing back one;
            # - you may find it easiest to initially append `None` elements to
            #   `parent_nodes` and to mutate them once their parent is inserted;
            # - feel free to use a different approach if you find it easier to do so.


        self.parent_nodes = parent_nodes

        
    def __getitem__(self, symbol):
        """Encode a single symbol.

        Args:
            symbol (int): A symbol from the alphabet {0, 1, ..., len-1}.
                
        Returns:
            The code word for `symbol`.
        """
        assert 0 <= symbol and symbol < len(self)
        
        codeword_reverse = []
        index = symbol
        while True:
            parent = self.parent_nodes[index]
            if parent is None:
                # Found root node.
                break
            else:
                index, bit = parent
                codeword_reverse.append(bit)
        
        return list(reversed(codeword_reverse))
    
    def __len__(self):
        """Returns the size of the alphabet."""
        # A binary tree with `N` leaves has `M = 2*N - 1` nodes, regardless of
        # its shape. Therefore, `N = (M + 1) / 2`.
        return (len(self.parent_nodes) + 1) // 2

### Unit Tests

In [3]:
def minimal_test():
    probabilities = [.3, .28, .12, .1, .2]
    expected_codebook = {
        0: [True, True],
        1: [True, False],
        2: [False, True, True],
        3: [False, True, False],
        4: [False, False],
    }
    codebook = Huffman(probabilities)
    for symbol, expected_codeword in expected_codebook.items():
        assert codebook[symbol] == expected_codeword

minimal_test()

In [4]:
# TODO: implement more tests, in particular for edge cases
def edge_case():
    probabilities = [.3, .28, .12, .1, .2, 0]
    expected_codebook = {
        0: [True, True],
        1: [True, False],
        2: [False, True, True],
        3: [False, True, False],
        4: [False, False],
    }
    codebook = Huffman(probabilities)
    for i, _ in enumerate(probabilities):
        print(i, codebook[i])


# if some probs are 0, we see that the codebook is not prefix free anymore
edge_case()

0 [True, True]
1 [True, False]
2 [False, True, True]
3 [False, True, False]
4 [False, False]
5 [False]


AssertionError: 